In [1]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
pos_vectors = KeyedVectors.load_word2vec_format('../pos_naver_news.vec', binary=False)


C:\Users\kvtor\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import json
import numpy as np
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math

l_quality= open("lecture_quality.txt", "r", encoding="utf-8").readlines()
load= open("load.txt", "r", encoding="utf-8").readlines()
grade= open("grade.txt", "r", encoding="utf-8").readlines()
int_friendly= open("international_friendly.txt", "r", encoding="utf-8").readlines()
CATEGORY_COUNT = 3

In [3]:
scores = []
subjects = [l_quality, load, grade]
def add_to_dict(subject):
    subject_scores={}
    for i in range(len(subject)):
        p = subject[i].split("\t")
        subject_scores[p[0]] = float(p[1][:-1])
    scores.append(subject_scores)

for subject in subjects:
    add_to_dict(subject)

In [4]:
subject_mp = {"강의":0,"널널":1,"성적":2}
def calculate_score(s, subject_str):
    result = 0
    for k,v in scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

In [5]:
REVIEWS = "cs_reviews"

f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

scores_list = []
subjects_str = ["강의", "널널", "성적"]
for subject_str in subjects_str:
    scores_list_subject=[]
    for i in range(len(f[:])):
        line = f[i]
        course = json.loads(line)
        scores_list_subject.append(calculate_score(course["review_text"], subject_str))
    scores_list.append(scores_list_subject)


In [6]:
f_count_word = {}
count_word = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}

def calc_weight(s):
    return 3 - (mp[s])

for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] == "?":
        continue
    words = course["review_text"].split()
    for w in words:
        if w not in count_word:
            count_word[w] = 0
        count_word[w] += calc_weight(course["강의"])


In [7]:
sorted_by_value = sorted(count_word.items(), key=lambda kv: kv[1])[::-1]

In [8]:
# sorted_by_value

In [9]:
score_sum=[0]*7
numberof = [0]*7

def calc_score_avg(column_name):
    for i in range(len(f[:])):
        course = json.loads(f[i])
        #print(mp[course["강의"]],(scores_list[i]))
        if course[column_name] == "?":
            continue
        score_sum[mp[course[column_name]]] += scores_list[subject_mp[column_name]][i]
        numberof[mp[course[column_name]]] += 1;
    

for x in subjects_str:
    print(x)
    score_sum=[0 for j in range(7)]
    numberof = [0 for j in range(7)]
    calc_score_avg(x)
    for k,v in mp.items():
        print(k,score_sum[v]/numberof[v])
    print("-----------------------")


강의
A 0.11654140171441107
B -0.11605949892652138
C -0.14389836361025826
D -0.3468267232471922
F -0.35336293460933754
-----------------------
널널
A 0.09717938413993184
B -0.04650993923502493
C -0.2622337094914021
D -0.33851569397971554
F -0.4073086596863776
-----------------------
성적
A -0.005961045765132882
B -0.10522973634678758
C -0.14126067112300067
D -0.1548596912691002
F -0.028196474317843428
-----------------------


In [10]:
def append_noun(s):
    return "('%s','Noun')" % (s)

naver_wv_categories = ["강의", "부담", "성적"]
wv_update = {}
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        if append_noun(k) in pos_vectors.vocab:
#             print(k,naver_wv_categories[i])
#             print(pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i])))
            wv_update[k] = pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i]))
        

In [17]:
# wv_update

In [12]:
subjects_str

['강의', '널널', '성적']

In [13]:
review_update = {}
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)

    for i in range(CATEGORY_COUNT):
        if course[subjects_str[i]] == "?":
            continue
        for k,v in scores[i].items():
            if k in course["review_text"].split():
                if k not in review_update:
                    review_update[k] = 0
                review_update[k] += calc_weight(course[subjects_str[i]])*100.0/len(course["review_text"])
        

In [18]:
# review_update

In [15]:
aggregated_scores = [{} for i in range(CATEGORY_COUNT)]
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        aggregated_scores[i][k] = v
        if k in review_update:
            aggregated_scores[i][k] += review_update[k]*0.1
        
        if k in wv_update:
            aggregated_scores[i][k] += wv_update[k]
            
            

In [16]:
def calc_aggregated_score(s, subject_str):
    result = 0
    for k,v in aggregated_scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

def calc_aggregated_score_sum(column_name):
    for i in range(len(f[:])):
        course = json.loads(f[i])
        if course[column_name] == "?":
            continue
        score_sum[mp[course[column_name]]] += calc_aggregated_score(course["review_text"], column_name)
        numberof[mp[course[column_name]]] += 1;

for x in subjects_str:
    print(x)
    score_sum=[0 for j in range(7)]
    numberof = [0 for j in range(7)]
    calc_aggregated_score_sum(x)
    for k,v in mp.items():
        print(k,score_sum[v]/numberof[v])
    print("-----------------------")

강의
A 0.22922615696817022
B 0.05405823837434913
C 0.04765070303320543
D -0.11107667974773391
F -0.16637902913740582
-----------------------
널널
A -0.18696934691497555
B -0.23019462132259694
C -0.36811701323245655
D -0.336929598405219
F -0.4729440046232905
-----------------------
성적
A -0.062479484600922085
B -0.10389359565606517
C -0.1409986103312977
D -0.16375215974264584
F 0.08446191015236892
-----------------------
